In [3]:
!pip install snowflake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 8.2 MB/s eta 0:00:00


In [21]:
import snowflake.connector
import pandas as pd

conn = snowflake.connector.connect(
    user='vaspon2005',
    password='Roarriserevolt2022',
    account='DXFYKFD-HJ95185',
    warehouse='COMPUTE_WH',
    database='SNOWFLAKE_SAMPLE_DATA',
    schema='TPCH_SF1'
)


In [22]:
sql = """
SELECT
    C.C_CUSTKEY,
    C.C_NAME,
    C.C_MKTSEGMENT,
    COALESCE(COUNT(O.O_ORDERKEY), 0) AS NUM_ORDERS,
    COALESCE(AVG(O.O_TOTALPRICE), 0) AS AVG_ORDER_VALUE,
    MAX(O.O_ORDERDATE) AS LAST_ORDER_DATE,
    COALESCE(SUM(O.O_TOTALPRICE), 0) AS TOTAL_SPENT
FROM CUSTOMER C
LEFT JOIN ORDERS O ON C.C_CUSTKEY = O.O_CUSTKEY
GROUP BY C.C_CUSTKEY, C.C_NAME, C.C_MKTSEGMENT

"""
df = pd.read_sql(sql, conn)




<ipython-input-22-866f5da8272d>:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [23]:
df_encoded = pd.get_dummies(df, columns=['C_MKTSEGMENT'], drop_first=True)

X = df_encoded.drop(columns=['C_CUSTKEY', 'C_NAME', 'LAST_ORDER_DATE', 'TOTAL_SPENT'])
y = df_encoded['TOTAL_SPENT']


In [14]:
from snowflake.connector.pandas_tools import write_pandas

create_query = """
CREATE TABLE CUSTOMER_FEATURES (
    C_CUSTKEY INTEGER,
    C_NAME STRING,
    NUM_ORDERS INTEGER,
    TOTAL_SPENT FLOAT,
    AVG_ORDER_VALUE FLOAT,
    TOTAL_QUANTITY FLOAT,
    AVG_DISCOUNT FLOAT,
    LAST_ORDER_DATE DATE,
    DAYS_SINCE_LAST_ORDER INTEGER
);
"""
conn.cursor().execute(create_query)

# Uploading to Snowflake
success, nchunks, nrows, _ = write_pandas(conn, df_features, table_name='CUSTOMER_FEATURES')
print(f"Inserted {nrows} rows into Snowflake feature store.")


ProgrammingError: 001003 (42000): SQL compilation error:
syntax error line 1 at position 0 unexpected 'REPLACE'.

In [15]:
query_features = "SELECT * FROM CUSTOMER_FEATURES"
df_model = pd.read_sql(query_features, conn)

X = df_model.drop(columns=['C_CUSTKEY', 'C_NAME', 'LAST_ORDER_DATE'])

y = X['TOTAL_SPENT']

X = X.drop(columns=['TOTAL_SPENT'])


KeyError: "['LAST_ORDER_DATE'] not found in axis"

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Linear Regression MSE:", round(mse, 2))
print("Linear Regression R² Score:", round(r2, 2))

Linear Regression MSE: 66587586863.27
Linear Regression R² Score: 0.96
